# Project Scheduling
## Calendar problem - scheduled breaks

For this example, we are going to create a basic project scheduling model. We will take into account worker assignments and costs.

We consider the following instance:
- We have a project with a list of tasks that need to be performed 
- Each task has requirements to be fulfilled by the available workers
- Each worker has different skills and an expertise level needed to fulfill the requirements
- Each worker has a fixed and a variable cost for performing a requirement
- Tasks have precedence behavior; some tasks cannot start before another one starts or ends
- Parent tasks are defined, a parent tasks define the span of a group of tasks
- Some workers have scheduled breaks that must be respected

## IBM Watson Studio

IBM Watson Studio is a platform software for data science. The advantage of using Watson is having a workspace that includes multiple collaboration and open-source tools such as RStudio, Spark and Python, all in an integrated environment. 

As you create a new project, you can add multiple collaborators, all having access to various predictive and prescriptive analytics models. Data can be accessed through Watson Data Platform, on-premise or on the cloud.

For this example, we are creating a new project, loading data files and creating a Decision Optimization (DO) Experiment.

## Source code

In [Assign workers - watson instructions](https://github.com/SmartBP/Modeling_Training/blob/master/Project%20Scheduling/Watson/Assign%20workers%20-%20watson%20instructions.ipynb) you can find step-to-step instructions to create a project and a DO experiment to solve the project scheduling problems. In this notebook you will find the OPL code for the optimization model and the JSON file script to create the gantt diagram.

### OPL code 

Here you can find the source code for the calendar problem - it takes into account scheduled breaks for the workers.


In [ ]:

using CP;
 
/********
 * Data *
 ********/

tuple Task {
  key int id;
  string  name;
  int     ptMin; // minimum duration
};

{ Task } Tasks = ...;

tuple hierarchy_data {  // hierarchy data
  int taskId;
  int parentId;
};

{ hierarchy_data } Hierarchy = ...;
{ int } Parents = { p.parentId | p in Hierarchy };

tuple Precedence {
  int    beforeId;
  int    afterId;
  string type;
  int    delay;
};

{ Precedence } Precedences = ...;

tuple Worker {
  key int id;
  string  name;
  float   fixedCost;
  float   varCost;
};

{ Worker } Workers = ...;

tuple Skill {
  key int id;
  string  name;
};

{ Skill } Skills = ...;

tuple Proficiency {
  int workerId;
  int skillId;
  int level;
};
  
{ Proficiency } Proficiencies = ...;

tuple Requirement {
  key int id;
  int     taskId;
};

{ Requirement } Requirements = ...;

tuple RequiredSkill {
  int reqId;
  int skillId;
  int levelMin;
  int levelMax;
};

{ RequiredSkill } RequiredSkills = ...;

{ int } PossibleWorkers[r in Requirements] = 
   { p.workerId | p in Proficiencies, n in RequiredSkills : 
     (n.reqId==r.id) && 
     (p.skillId==n.skillId) && 
     (n.levelMin <= p.level) && 
     (p.level <= n.levelMax) };

tuple Alloc {
  int reqId;
  int workerId;
  int pt;
};

{ Alloc } Allocations = { <r.id, i, t.ptMin> | r in Requirements, t in Tasks, i in PossibleWorkers[r] : t.id==r.taskId };

// KPIs
tuple kpi{
  string kpi;
  float value;  
}

{kpi} KPIs = ...;
{string} kpi_name = {p | <p,v> in KPIs};
float kpi_value[kpi_name] = [p:v | <p,v> in KPIs];
float MakespanWeight = kpi_value["Makespan"];
float FixedWeight = kpi_value["WorkersFixed"];
float VariableWeight = kpi_value["WorkersProportional"];

int MaxInterval = 10000001;

execute CPX_PARAMS{
  cp.param.timeLimit = 60;
  cp.param.TemporalRelaxation = "Off";
};

/**********************
 * Decision variables *
 **********************/
 
dvar interval task[t in Tasks] size t.ptMin..MaxInterval;
dvar interval alts[a in Allocations] optional;
dvar interval workerSpan[w in Workers] optional;

/************************
 * Decision expressions *
 ************************/
 
dexpr int makespan = max(t in Tasks) endOf(task[t]);
dexpr int workerTimeSpent[w in Workers] = sum(a in Allocations: a.workerId==w.id) sizeOf(alts[a],0);
dexpr float workersFixedCost = sum(w in Workers) w.fixedCost * presenceOf(workerSpan[w]); 
dexpr float workersVariableCost = sum(w in Workers) w.varCost*workerTimeSpent[w];

/*************
 * Objective *
 *************/
 
minimize MakespanWeight * makespan 
         + FixedWeight * workersFixedCost 
         + VariableWeight * workersVariableCost;

/***************
 * Constraints *
 ***************/
 
subject to {   
  // Hierarchy structure
  forall (t in Tasks : t.id in Parents)
       span(task[t], all(i in Hierarchy: i.parentId == t.id) task[<i.taskId>]); 
       
   // Precedence constraints
  forall (p in Precedences : p.type == "StartsAfterStart") 
      startBeforeStart(task[<p.beforeId>], task[<p.afterId>], p.delay);
  forall (p in Precedences : p.type == "StartsAfterEnd") 
      endBeforeStart(task[<p.beforeId>], task[<p.afterId>], p.delay);
  
  // Alternatives of (each requirement must be perfomed by one worker)
  forall(r in Requirements)
    alternative(task[<r.taskId>], all(a in Allocations: a.reqId==r.id) alts[a]);

  // A worker can perform only one task requirement at any point in time
  forall(w in Workers)
    noOverlap(all(a in Allocations: a.workerId==w.id) alts[a]);
    
  // Calculate whether each worker is used in the project
  forall(w in Workers)
    span(workerSpan[w], all(a in Allocations: a.workerId==w.id) alts[a]);
    	
};

tuple result_task{
   int taskId;
   int start;
   int end; 
} 

{result_task} results_tasks = {<t.id, startOf(task[t]), endOf(task[t])> | t in Tasks};

tuple result_alloc{
   int reqId;
   int taskId;
   int workerId;
   int start;
   int end; 
} 

{result_alloc} results_allocs = {<a.reqId, r.taskId, a.workerId, startOf(alts[a]), endOf(alts[a])> | r in Requirements, a in Allocations: r.id == a.reqId};

tuple result_worker{
   int workerId;
   string workerName;
   int start;
   int end; 
} 

{result_worker} results_workers = {<w.id, w.name, startOf(workerSpan[w]), endOf(workerSpan[w])> | w in Workers};

tuple result_task_date{
   int taskId;
   string start;
   string end; 
} 
{result_task_date} result_task_dates = {};

tuple result_alloc_date{
   int reqId;
   int taskId;
   int workerId;
   string start;
   string end; 
}
{result_alloc_date} result_alloc_dates = {};

tuple result_worker_date{
   int workerId;
   string workerName;
   string start;
   string end; 
}
{result_worker_date} result_worker_dates = {};

tuple workerTime{
  int workerId;
  string workerName;
  int duration;
}

{workerTime} workerTimes = {<w.id,w.name,workerTimeSpent[w]>|w in Workers};

execute{
    for(var t in results_tasks){
        var start = t.start;
        var end = t.end;
        var start_date = new Date(2021, 0, 1, 0, start, 0, 0);
        var end_date = new Date(2021, 0, 1, 0, end, 0, 0);
        result_task_dates.add(t.taskId, start_date.toString(), end_date.toString());
    }

    for(var t in results_allocs){
        var start = t.start;
        var end = t.end;
        if(start+end > 0){
            var start_date = new Date(2021, 0, 1, 0, start, 0, 0);
            var end_date = new Date(2021, 0, 1, 0, end, 0, 0);
            result_alloc_dates.add(t.reqId, t.taskId, t.workerId, start_date.toString(), end_date.toString());
        }
    } 

    for(var t in results_workers){
        var start = t.start;
        var end = t.end;
        if(start+end > 0){
            var start_date = new Date(2021, 0, 1, 0, start, 0, 0);
            var end_date = new Date(2021, 0, 1, 0, end, 0, 0);
            result_worker_dates.add(t.workerId, t.workerName, start_date.toString(), end_date.toString());
        }
    } 
};


### JSON file
Here you can find the JSON file to create the gantt diagram in watson

In [ ]:
{
  "name": "",
  "type": "Gantt",
  "props": {
    "container": "",
    "data": [
      "Workers",
      "result_alloc_dates"
    ],
    "spec": {
      "resources": {
        "data": "Workers",
        "id": "id",
        "name": "name"
      },
      "activities": {
        "data": "result_alloc_dates",
        "id": "reqId",
        "name": "taskId",
        "start": "start",
        "end": "end"
      },
      "reservations": {
        "data": "result_alloc_dates",
        "activity": "reqId",
        "resource": "workerId"
      },
      "constraints": {},
      "dateFormat": "MM/dd/yyyy HH:mm:ss SSS"
    },
    "search": ""
  }
}